**Use GPU: Runtime -> Change runtime type -> GPU (Hardware Accelerator)**

Setup

In [1]:
!cat ~/.keras/keras.json

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

Mini-VGGNet

In [2]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [3]:
class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

Train MiniVGGNet w/ Checkpoints (Improvements)

In [4]:
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.datasets import cifar10
import os

In [5]:
def train_minivggnet_incr_checkpoints(save_filepath):
    # load the training and testing data, then scale it into the
    # range [0, 1]
    print("[INFO] loading CIFAR-10 data...")
    ((trainX, trainY), (testX, testY)) = cifar10.load_data()
    trainX = trainX.astype("float") / 255.0
    testX = testX.astype("float") / 255.0

    # convert the labels from integers to vectors
    lb = LabelBinarizer()
    trainY = lb.fit_transform(trainY)
    testY = lb.transform(testY)

    # initialize the optimizer and model
    print("[INFO] compiling model...")
    opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
    model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    # construct the callback to save only the *best* model to disk
    # based on the validation loss
    fname = os.path.sep.join([save_filepath, "minivggnet-weights-{epoch:03d}-{val_loss:.4f}.hdf5"])
    checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
    callbacks = [checkpoint]

    # train the network
    print("[INFO] training network...")
    H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=40, callbacks=callbacks, verbose=2)

In [6]:
train_minivggnet_incr_checkpoints(save_filepath="drive/MyDrive/pyimagesearch/output/checkpoints")

[INFO] loading CIFAR-10 data...
170500096/170498071 [==============================] - 4s 0us/step
[INFO] compiling model...
[INFO] training network...
Epoch 1/40
782/782 - 39s - loss: 1.5947 - accuracy: 0.4643 - val_loss: 1.3486 - val_accuracy: 0.5288

Epoch 00001: val_loss improved from inf to 1.34858, saving model to drive/MyDrive/pyimagesearch/checkpoints/minivggnet-weights-001-1.3486.hdf5
Epoch 2/40
782/782 - 6s - loss: 1.1347 - accuracy: 0.6023 - val_loss: 0.9045 - val_accuracy: 0.6834

Epoch 00002: val_loss improved from 1.34858 to 0.90451, saving model to drive/MyDrive/pyimagesearch/checkpoints/minivggnet-weights-002-0.9045.hdf5
Epoch 3/40
782/782 - 6s - loss: 0.9623 - accuracy: 0.6636 - val_loss: 0.8380 - val_accuracy: 0.7046

Epoch 00003: val_loss improved from 0.90451 to 0.83800, saving model to drive/MyDrive/pyimagesearch/checkpoints/minivggnet-weights-003-0.8380.hdf5
Epoch 4/40
782/782 - 6s - loss: 0.8544 - accuracy: 0.6995 - val_loss: 0.7380 - val_accuracy: 0.7394

Epoch 

Train MiniVGGNet w/ Checkpoints (Best)

In [7]:
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.datasets import cifar10

In [8]:
def train_minivggnet_best_checkpoints(save_filepath):
    # load the training and testing data, then scale it into the
    # range [0, 1]
    print("[INFO] loading CIFAR-10 data...")
    ((trainX, trainY), (testX, testY)) = cifar10.load_data()
    trainX = trainX.astype("float") / 255.0
    testX = testX.astype("float") / 255.0

    # convert the labels from integers to vectors
    lb = LabelBinarizer()
    trainY = lb.fit_transform(trainY)
    testY = lb.transform(testY)

    # initialize the optimizer and model
    print("[INFO] compiling model...")
    opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
    model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

    # construct the callback to save only the *best* model to disk
    # based on the validation loss
    checkpoint = ModelCheckpoint(save_filepath, monitor="val_loss", save_best_only=True, verbose=1)
    callbacks = [checkpoint]

    # train the network
    print("[INFO] training network...")
    H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=40, callbacks=callbacks, verbose=2)

In [9]:
train_minivggnet_best_checkpoints(save_filepath="drive/MyDrive/pyimagesearch/output/checkpoints")

[INFO] loading CIFAR-10 data...
[INFO] compiling model...
[INFO] training network...
Epoch 1/40
782/782 - 7s - loss: 1.6677 - accuracy: 0.4400 - val_loss: 1.1411 - val_accuracy: 0.5940

Epoch 00001: val_loss improved from inf to 1.14105, saving model to drive/MyDrive/pyimagesearch/checkpoints
INFO:tensorflow:Assets written to: drive/MyDrive/pyimagesearch/checkpoints/assets
Epoch 2/40
782/782 - 6s - loss: 1.1708 - accuracy: 0.5892 - val_loss: 0.9447 - val_accuracy: 0.6697

Epoch 00002: val_loss improved from 1.14105 to 0.94474, saving model to drive/MyDrive/pyimagesearch/checkpoints
INFO:tensorflow:Assets written to: drive/MyDrive/pyimagesearch/checkpoints/assets
Epoch 3/40
782/782 - 6s - loss: 0.9789 - accuracy: 0.6582 - val_loss: 0.8778 - val_accuracy: 0.6939

Epoch 00003: val_loss improved from 0.94474 to 0.87778, saving model to drive/MyDrive/pyimagesearch/checkpoints
INFO:tensorflow:Assets written to: drive/MyDrive/pyimagesearch/checkpoints/assets
Epoch 4/40
782/782 - 6s - loss: 0.